# Visualization

In [1]:
from brian2 import *
from brian2modelfitting import *

import numpy as np

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]
WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
 [py.warnings]


In [2]:
dt = 0.01*ms
defaultclock.dt = dt

## Load the Data

In [3]:
voltage1 = genfromtxt('voltage1nA.csv', delimiter=',')
voltage2 = genfromtxt('voltage2nA.csv', delimiter=',')

In [4]:
inp_cur1 = genfromtxt('inp_cur1nA.csv', delimiter=',')
inp_cur2 = genfromtxt('inp_cur2nA.csv', delimiter=',')

In [5]:
inp_current = np.array([inp_cur1, inp_cur2])
inp_current

array([[ 0.        , -0.01416314,  0.06263459, ..., -0.44478035,
        -0.39163954, -0.33014548],
       [ 0.        ,  0.39594954,  0.27878635, ..., -0.55344109,
        -0.71057181, -0.62256885]])

In [6]:
out_spikes1 = genfromtxt('out_spikes1nA.csv', delimiter=',')
out_spikes2 = genfromtxt('out_spikes2nA.csv', delimiter=',')
out_spikes = [out_spikes1, out_spikes2]
# out_spikes

## Define Model

In [7]:
res={'taut': 0.038281479669330376,
 'R': 8084051733.384311,
 'D': 0.0025722558443824674,
 'a': 0.08427460573972566,
 'tau': 0.01007468391773357,
 'alpha': 1.4880312994647356}



In [8]:
res={'taut': 0.03940264530424703,
 'tau': 0.019428036380643388,
 'alpha': 1.0694762113655902,
 'R': 13154191074.086367,
 'a': 0.09021720088860954,
 'D': 0.00047695430996046867}

In [ ]:

model = '''
        dv/dt = (R*I(t, i)- v)/tau : 1
        dvt/dt = (a*v - vt) / taut :1
        alpha : 1 (constant)
        a : 1 (constant)
        R : 1/amp (constant)
        tau : second (constant)
        taut : second (constant)
        D : second(constant)
        '''

reset = '''
v = 0
vt = vt + alpha
'''

In [ ]:
I = TimedArray(inp_current*nA, dt=dt)

In [ ]:
start_scope()
G = NeuronGroup(2, model,
                threshold='v > 1 + vt',
                reset=reset,
                refractory='D',
                method='exponential_euler')

G.set_states(res, units=False)
mon = StateMonitor(G, 'v', record=True)
spike_mon = SpikeMonitor(G)

In [ ]:
run(20*second)

In [ ]:
fits = mon.v[:]
spikes = getattr(spike_mon, 't') / ms

In [ ]:
t0 = np.arange(0, len(voltage1))*dt/ms
t_ = spikes[0]
t = spikes[0][t_<2000]

In [ ]:
v = fits
d = 2000000

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(14,10))
ax[0].plot(t0[:d]/ms, inp_cur1[:d]);
ax[1].plot(t0[:d]/ms, voltage1[:d]);
ax[2].plot(np.arange(0, len(v))*dt/ms, v,'g');
ax[2].vlines(spikes, v[np.int_(np.round(spikes/dt*ms))]-60, v[np.int_(np.round(spikes/dt*ms))]+100, 'g');


In [ ]:
# for i in range(0, 199999, 5000):
#     print(i)

In [ ]:
plt.figure(figsize=(30,5))
plot(t0[:d], voltage2[:d]);
# plot(np.arange(0, len(v))*dt/ms, v-75,'r');
vlines(spikes, v[np.int_(np.round(spikes/dt*ms))]-75, v[np.int_(np.round(spikes/dt*ms))]+60, 'r');
xlim(2500, 4000)

In [ ]:
plt.figure(figsize=(10,5))
plot(np.arange(0, len(v))*dt/ms, v-60,'g');
vlines(spikes, v[np.int_(np.round(spikes/dt*ms))]-60, v[np.int_(np.round(spikes/dt*ms))]+100, 'g');
